<a href="https://colab.research.google.com/github/chuviethongfpt/atom-assignments/blob/main/HOME_ASSIGNMENT_3_SLACK_API_TO_GSHEET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mục đích của bài Assignment**


*   Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
*   **[Optional 1]** Đưa danh sách Users lên Google Spreadsheet, để theo dõi
*   **[Optional 2]** Lấy thông tin Assignment Submission và số Reviews trên #atom-assignmentnt2 và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng** 

*   Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
*   Trích xuất thông tin từ JSON
*   Dùng module gspread để đưa thông tin lên Google Spreadsheet

# 0.Load Modules

In [10]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

# 1.Slack API: User List¶




In [15]:
!ls

 env_variable.json   sample_data  'Token generate.json'


In [16]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [17]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN']

In [18]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

# TODO #1

In [36]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [37]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [38]:
user_df[user_df.display_name == 'Việt Hồng Chu'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
34,U01UMC0N5U2,chuviethong.vn,Việt Hồng Chu,Việt Hồng Chu,Business Analyst,0899306779,False


# Optional 1: Update data => Google Spreadsheet

# TODO#2

In [47]:
!ls

 env_variable.json			    sample_data
 fresh-geography-314306-c804644208a6.json  'Token generate.json'


In [48]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'fresh-geography-314306-c804644208a6.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [51]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1XFv2QcX0u8VpxdR-0OeMeWHQmxwF3vzgrOem67AgASE' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

APIError: ignored